In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load the trained model artifacts
rf_model = joblib.load('random_forest_model.pkl')
scaler = joblib.load('scaler.pkl')
label_encoders = joblib.load('label_encoders.pkl')

# Default values and ranges
default = {
    'Product_Category': 'Food',
    'Shipping_Mode': 'Rail',
    'Dominant_Buyer_Flag': 0,
    'Quantity_Ordered': 365,
    'Order_Value_USD': 7154.54,
    'Historical_Disruption_Count': 19,
    'Supplier_Reliability_Score': 0.8,
    'Available_Historical_Records': 9000
}
ranges = {
    'Quantity_Ordered': (0, 1000, 1),
    'Order_Value_USD': (0.0, 50000.0, 100.0),
    'Historical_Disruption_Count': (0, 50, 1),
    'Supplier_Reliability_Score': (0.0, 1.0, 0.01),
    'Available_Historical_Records': (0, 20000, 100)
}

# Sliders and dropdowns
sliders = {
    k: widgets.FloatSlider(value=default[k], min=ranges[k][0], max=ranges[k][1], step=ranges[k][2], description=k)
    for k in ['Supplier_Reliability_Score', 'Order_Value_USD']
}
sliders.update({
    k: widgets.IntSlider(value=default[k], min=ranges[k][0], max=ranges[k][1], step=ranges[k][2], description=k)
    for k in ['Quantity_Ordered', 'Historical_Disruption_Count', 'Available_Historical_Records']
})
sliders['Dominant_Buyer_Flag'] = widgets.IntSlider(value=default['Dominant_Buyer_Flag'], min=0, max=1, step=1, description='Dominant_Buyer_Flag')
category_dropdown = widgets.Dropdown(options=label_encoders['Product_Category'].classes_, value=default['Product_Category'], description='Product_Category')
mode_dropdown = widgets.Dropdown(options=label_encoders['Shipping_Mode'].classes_, value=default['Shipping_Mode'], description='Shipping_Mode')
feature_to_vary_dropdown = widgets.Dropdown(options=list(default.keys()), value='Supplier_Reliability_Score', description='Vary Feature')

# Output widget for risk flag
out_flag = widgets.Output()

def plot_and_predict(feature_to_vary, Product_Category, Shipping_Mode, Dominant_Buyer_Flag,
                     Quantity_Ordered, Order_Value_USD, Historical_Disruption_Count,
                     Supplier_Reliability_Score, Available_Historical_Records):
    # Vary chosen feature, keep others fixed
    min_val, max_val, step = ranges[feature_to_vary]
    values = np.arange(min_val, max_val + step, step)
    base_sample = {
        'Product_Category': Product_Category,
        'Shipping_Mode': Shipping_Mode,
        'Dominant_Buyer_Flag': Dominant_Buyer_Flag,
        'Quantity_Ordered': Quantity_Ordered,
        'Order_Value_USD': Order_Value_USD,
        'Historical_Disruption_Count': Historical_Disruption_Count,
        'Supplier_Reliability_Score': Supplier_Reliability_Score,
        'Available_Historical_Records': Available_Historical_Records
    }
    probs = []
    for val in values:
        sample = base_sample.copy()
        sample[feature_to_vary] = val
        df_sample = pd.DataFrame([sample])
        categorical_cols = ['Product_Category', 'Shipping_Mode']
        for col in categorical_cols:
            le = label_encoders[col]
            df_sample[col] = df_sample[col].apply(lambda x: x if x in le.classes_ else 'None')
            if 'None' not in le.classes_:
                le.classes_ = np.append(le.classes_, 'None')
            df_sample[col + '_encoded'] = le.transform(df_sample[col])
            df_sample.drop(columns=[col], inplace=True)
        expected_features = list(scaler.feature_names_in_)
        for col in expected_features:
            if col not in df_sample.columns:
                df_sample[col] = 0
        df_sample = df_sample[expected_features]
        X_scaled = scaler.transform(df_sample)
        prob = rf_model.predict_proba(X_scaled)[:, 1][0]
        probs.append(prob)
    plt.figure(figsize=(8,5))
    sns.lineplot(x=values, y=probs)
    plt.xlabel(feature_to_vary)
    plt.ylabel('Predicted Probability of Risk')
    plt.title(f'Effect of {feature_to_vary} on Predicted Risk')
    plt.grid(True)
    plt.show()
    # Also predict at current slider settings for binary risk flag
    sample_now = pd.DataFrame([base_sample])
    for col in categorical_cols:
        le = label_encoders[col]
        sample_now[col] = sample_now[col].apply(lambda x: x if x in le.classes_ else 'None')
        if 'None' not in le.classes_:
            le.classes_ = np.append(le.classes_, 'None')
        sample_now[col + '_encoded'] = le.transform(sample_now[col])
        sample_now.drop(columns=[col], inplace=True)
    for col in expected_features:
        if col not in sample_now.columns:
            sample_now[col] = 0
    sample_now = sample_now[expected_features]
    X_now_scaled = scaler.transform(sample_now)
    flag = rf_model.predict(X_now_scaled)[0]
    with out_flag:
        clear_output(wait=True)
        print(f"Current slider settings: Predicted SUPPLY RISK FLAG = {flag}")

ui = widgets.VBox([
    feature_to_vary_dropdown,
    category_dropdown, mode_dropdown,
    sliders['Dominant_Buyer_Flag'],
    sliders['Quantity_Ordered'],
    sliders['Order_Value_USD'],
    sliders['Historical_Disruption_Count'],
    sliders['Supplier_Reliability_Score'],
    sliders['Available_Historical_Records']
])
out = widgets.interactive_output(plot_and_predict, {
    'feature_to_vary': feature_to_vary_dropdown,
    'Product_Category': category_dropdown,
    'Shipping_Mode': mode_dropdown,
    'Dominant_Buyer_Flag': sliders['Dominant_Buyer_Flag'],
    'Quantity_Ordered': sliders['Quantity_Ordered'],
    'Order_Value_USD': sliders['Order_Value_USD'],
    'Historical_Disruption_Count': sliders['Historical_Disruption_Count'],
    'Supplier_Reliability_Score': sliders['Supplier_Reliability_Score'],
    'Available_Historical_Records': sliders['Available_Historical_Records']
})
display(ui, out, out_flag)


Output()

Output()